# Assignment 3: Ensemble models lab
---
Members
1. Kanyaluck Chimchome (64070501003)
2. Boonyarit Samran (64070501028)
3. Warisara Patib (64070501044)
4. Chanidapa Chanama (64070501090)
5. Nontawat Kunlayawuttipong (64070501093)
6. Nuttanon Yolnawa (64070507205)

In [2]:
import warnings; 
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from scipy import stats

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from platform import python_version


print('Python version', python_version())
print('Numpy version', np.__version__)
print('Scipy version', sp.__version__)
print('Pandas version', pd.__version__)
print('Matplotlib version', mpl.__version__)
###############################################

#plt.style.use('ggplot')
plt.style.use('seaborn-muted')
plt.rcParams['figure.figsize'] = (6, 6)
plt.rcParams['grid.linestyle'] = ':'   
plt.rcParams['axes.grid'] = False


%matplotlib inline   
%config InlineBackend.figure_formats = {'png', 'retina'}
%load_ext autoreload
%autoreload 2

Python version 3.11.4
Numpy version 1.26.0
Scipy version 1.11.2
Pandas version 2.1.1
Matplotlib version 3.8.0


OSError: 'seaborn-muted' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

# <font color='orange'>Loan prediction -- Data preparation</font>

## Load and explore the data set

In [4]:
loan_df = pd.read_csv('data/loans/Training Data.csv')

Look at the first few rows of the dataset

In [5]:
loan_df.head()

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


Check the summary information about the data set.
- How many samples are there ?
- Are there columns with missing values ?

In [6]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Id                 252000 non-null  int64 
 1   Income             252000 non-null  int64 
 2   Age                252000 non-null  int64 
 3   Experience         252000 non-null  int64 
 4   Married/Single     252000 non-null  object
 5   House_Ownership    252000 non-null  object
 6   Car_Ownership      252000 non-null  object
 7   Profession         252000 non-null  object
 8   CITY               252000 non-null  object
 9   STATE              252000 non-null  object
 10  CURRENT_JOB_YRS    252000 non-null  int64 
 11  CURRENT_HOUSE_YRS  252000 non-null  int64 
 12  Risk_Flag          252000 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 25.0+ MB


In [7]:
loan_df.isna().sum()

Id                   0
Income               0
Age                  0
Experience           0
Married/Single       0
House_Ownership      0
Car_Ownership        0
Profession           0
CITY                 0
STATE                0
CURRENT_JOB_YRS      0
CURRENT_HOUSE_YRS    0
Risk_Flag            0
dtype: int64

Sample only 100,000 rows from the dataframe to work with.

In [8]:
loan_sample = loan_df.sample(n=100000,random_state=1)
loan_sample

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
242744,242745,7765404,55,8,single,norent_noown,no,Lawyer,Kolhapur,Maharashtra,5,12,1
177358,177359,4161972,41,16,single,owned,no,Civil_engineer,Hajipur[31],Bihar,9,11,0
22218,22219,7767858,29,0,married,rented,no,Physician,Yamunanagar,Haryana,0,10,0
13974,13975,2135764,35,19,single,rented,no,Flight_attendant,Gopalpur,West_Bengal,5,14,0
193370,193371,3990809,58,12,single,owned,no,Mechanical_engineer,Mehsana,Gujarat,12,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206176,206177,3679290,25,9,single,rented,no,Psychologist,Pune,Maharashtra,6,11,0
184160,184161,6251533,50,16,single,rented,yes,Design_Engineer,Adoni,Andhra_Pradesh,4,11,0
40039,40040,415703,56,5,single,rented,no,Artist,Bellary,Karnataka,5,11,0
17063,17064,4816663,40,18,single,rented,no,Architect,Amroha,Uttar_Pradesh,3,14,0


Take a quick look at the numeric column statistics.  
Is there anything unusual?

In [9]:
loan_sample.describe(include="number")

,Id,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
count,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,125888.918610,4.996131e+06,49.903870,10.073060,6.331600,12.000090,0.123090
std,72866.381363,2.879613e+06,17.049303,6.009601,3.648607,1.399503,0.328542
min,2.000000,1.031000e+04,21.000000,0.000000,0.000000,10.000000,0.000000
25%,62943.500000,2.495998e+06,35.000000,5.000000,3.000000,11.000000,0.000000
50%,125525.500000,5.000063e+06,50.000000,10.000000,6.000000,12.000000,0.000000
75%,189231.500000,7.475446e+06,65.000000,15.000000,9.000000,13.000000,0.000000
max,251997.000000,9.999180e+06,79.000000,20.000000,14.000000,14.000000,1.000000


We should drop ID Column because it not numeric value

Take a quick look at the categorical column statistics.  
Are there any columns that you might remove?

In [10]:
loan_sample.dtypes

Id                    int64
Income                int64
Age                   int64
Experience            int64
Married/Single       object
House_Ownership      object
Car_Ownership        object
Profession           object
CITY                 object
STATE                object
CURRENT_JOB_YRS       int64
CURRENT_HOUSE_YRS     int64
Risk_Flag             int64
dtype: object

In [11]:
loan_sample.select_dtypes(include=['object'])

,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE
242744,single,norent_noown,no,Lawyer,Kolhapur,Maharashtra
177358,single,owned,no,Civil_engineer,Hajipur[31],Bihar
22218,married,rented,no,Physician,Yamunanagar,Haryana
13974,single,rented,no,Flight_attendant,Gopalpur,West_Bengal
193370,single,owned,no,Mechanical_engineer,Mehsana,Gujarat
...,...,...,...,...,...,...
206176,single,rented,no,Psychologist,Pune,Maharashtra
184160,single,rented,yes,Design_Engineer,Adoni,Andhra_Pradesh
40039,single,rented,no,Artist,Bellary,Karnataka
17063,single,rented,no,Architect,Amroha,Uttar_Pradesh


Look at column profiles using ydata_profiling  

In [14]:
import ydata_profiling
loan_sample.profile_report()

ModuleNotFoundError: No module named 'ydata_profiling'

## Data preprocessing

Duplicate the original dataframe.

In [15]:
loan_df_copy = loan_df.copy() 

List the column data types.

In [16]:
loan_df_copy.dtypes

Id                    int64
Income                int64
Age                   int64
Experience            int64
Married/Single       object
House_Ownership      object
Car_Ownership        object
Profession           object
CITY                 object
STATE                object
CURRENT_JOB_YRS       int64
CURRENT_HOUSE_YRS     int64
Risk_Flag             int64
dtype: object

Drop the columns `Id`, `Profession`, `CITY`, and `STATE` for now.  
To include `Profession` and `STATE` to the modeling, we could manually bin ```Profession``` into smaller   
    number of groups and ```STATE``` into regions.

In [17]:
loan_df_copy.drop(["Id","Profession","CITY","STATE"],axis=1,inplace=True)
loan_df_copy

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1303834,23,3,single,rented,no,3,13,0
1,7574516,40,10,single,rented,no,9,13,0
2,3991815,66,4,married,rented,no,4,10,0
3,6256451,41,2,single,rented,yes,2,12,1
4,5768871,47,11,single,rented,no,3,14,1
...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,single,rented,no,6,11,0
251996,2843572,26,10,single,rented,no,6,11,0
251997,4522448,46,7,single,rented,no,7,12,0
251998,6507128,45,0,single,rented,no,0,10,0


Look at unique values and counts of the remaining categorical columns.  

**Questions**<br>
- Which columns require label/ordinal encoding?  
- Which columns require one-hot encoding?

In [18]:
loan_df_copy.select_dtypes(include=['object'])

,Married/Single,House_Ownership,Car_Ownership
0,single,rented,no
1,single,rented,no
2,married,rented,no
3,single,rented,yes
4,single,rented,no
...,...,...,...
251995,single,rented,no
251996,single,rented,no
251997,single,rented,no
251998,single,rented,no


In [19]:
loan_df_copy.select_dtypes(include=['object']).apply(np.unique)

Married/Single                 [married, single]
House_Ownership    [norent_noown, owned, rented]
Car_Ownership                          [no, yes]
dtype: object

In [20]:
for i in loan_df_copy.select_dtypes(include=['object']).columns:
    loan_df_copy[i].value_counts()

Married/Single
single     226272
married     25728
Name: count, dtype: int64

House_Ownership
rented          231898
owned            12918
norent_noown      7184
Name: count, dtype: int64

Car_Ownership
no     176000
yes     76000
Name: count, dtype: int64

Apply label/ordinal encoding to columns (if any)

Apply one-hot encoding to column (if any)

In [21]:
loan_df_copy = pd.get_dummies(loan_df_copy, columns=["Married/Single","House_Ownership","Car_Ownership"])

In [22]:
loan_df_copy.head()

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single_married,Married/Single_single,House_Ownership_norent_noown,House_Ownership_owned,House_Ownership_rented,Car_Ownership_no,Car_Ownership_yes
0,1303834,23,3,3,13,0,False,True,False,False,True,True,False
1,7574516,40,10,9,13,0,False,True,False,False,True,True,False
2,3991815,66,4,4,10,0,True,False,False,False,True,True,False
3,6256451,41,2,2,12,1,False,True,False,False,True,False,True
4,5768871,47,11,3,14,1,False,True,False,False,True,True,False


In [23]:
loan_df_copy.dtypes

Income                          int64
Age                             int64
Experience                      int64
CURRENT_JOB_YRS                 int64
CURRENT_HOUSE_YRS               int64
Risk_Flag                       int64
Married/Single_married           bool
Married/Single_single            bool
House_Ownership_norent_noown     bool
House_Ownership_owned            bool
House_Ownership_rented           bool
Car_Ownership_no                 bool
Car_Ownership_yes                bool
dtype: object

In [24]:
loan_df_copy.drop(["Married/Single_single","Car_Ownership_yes"],axis=1,inplace=True)

In [25]:
loan_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   Income                        252000 non-null  int64
 1   Age                           252000 non-null  int64
 2   Experience                    252000 non-null  int64
 3   CURRENT_JOB_YRS               252000 non-null  int64
 4   CURRENT_HOUSE_YRS             252000 non-null  int64
 5   Risk_Flag                     252000 non-null  int64
 6   Married/Single_married        252000 non-null  bool 
 7   House_Ownership_norent_noown  252000 non-null  bool 
 8   House_Ownership_owned         252000 non-null  bool 
 9   House_Ownership_rented        252000 non-null  bool 
 10  Car_Ownership_no              252000 non-null  bool 
dtypes: bool(5), int64(6)
memory usage: 12.7 MB


Rename the target variable name to `Target`

In [26]:
loan_df_copy.rename(columns={"Risk_Flag":"Target"}, inplace=True)

In [27]:
loan_df_copy

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Target,Married/Single_married,House_Ownership_norent_noown,House_Ownership_owned,House_Ownership_rented,Car_Ownership_no
0,1303834,23,3,3,13,0,False,False,False,True,True
1,7574516,40,10,9,13,0,False,False,False,True,True
2,3991815,66,4,4,10,0,True,False,False,True,True
3,6256451,41,2,2,12,1,False,False,False,True,False
4,5768871,47,11,3,14,1,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,6,11,0,False,False,False,True,True
251996,2843572,26,10,6,11,0,False,False,False,True,True
251997,4522448,46,7,7,12,0,False,False,False,True,True
251998,6507128,45,0,0,10,0,False,False,False,True,True


70/30 train-test split 

In [28]:
X = loan_df_copy.drop(["Target"],axis=1)
y = loan_df_copy["Target"]

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# <font color='orange'>Random Forest</font>

**To do**:
- Fit random forest with default hyperparameters to the train data
- Generate the classification reports on the train and test data.
- Determine the f1-score by using K-fold CV on the train data.
- Tune the model hyperparameters.
- Determine the f1-score of the tuned model by using K-fold CV on the whole dataset. 

## Fit Random forest model

In [30]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,y_train)

RandomForestClassifier()

In [31]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Evaluate performance

### Train and test performance

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_train, clf.predict(X_train)))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96    154666
           1       0.72      0.80      0.76     21734

    accuracy                           0.94    176400
   macro avg       0.85      0.88      0.86    176400
weighted avg       0.94      0.94      0.94    176400



In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94     66338
           1       0.60      0.53      0.56      9262

    accuracy                           0.90     75600
   macro avg       0.77      0.74      0.75     75600
weighted avg       0.89      0.90      0.90     75600



### K-fold Cross-validation

In [34]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [49]:
scoring_metrics = ['f1']
scores = cross_validate(clf, X_train, y_train,
 scoring=scoring_metrics, cv = 5, return_train_score= False, n_jobs=-1)
scores_df = pd.DataFrame(scores)
scores_df
scores_df.mean()

,fit_time,score_time,test_f1
0,13.102246,0.575085,0.553212
1,13.008143,0.570551,0.561167
2,13.077186,0.577067,0.550280
3,13.203771,0.596309,0.574229
4,13.060287,0.566531,0.568442


fit_time      13.090327
score_time     0.577109
test_f1        0.561466
dtype: float64

## Model Tuning

- Perform Grid-search CV on `criterion`, `max_depth`, `max_features`, `n_estimators` by using F1
as the scoring metric.
- Show the tuned hyperparameter sets and the corresponding scores.
- Perform K-fold CV on the whole dataset using the tuned hyperparameters and compare the performance
results with the untuned model.

### Grid-search CV

In [37]:
# Specify the value of features to tune here.
param_grid = {
 'criterion': ["gini"],
 'max_depth': [5, 10 , 20, 30],
 'max_features': ["sqrt", 3, 6],
 'n_estimators': [70, 100],
}
dt_gridsearch = GridSearchCV(RandomForestClassifier(), param_grid, scoring='f1', verbose=True, n_jobs=-1)
dt_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [5, 10, 20, 30],
                         'max_features': ['sqrt', 3, 6],
                         'n_estimators': [70, 100]},
             scoring='f1', verbose=True)

In [38]:
pd.DataFrame(dt_gridsearch.cv_results_).sort_values('mean_test_score', ascending=False)
dt_gridsearch.best_params_ # Tuned hyperparameters
dt_gridsearch.best_score_ # Best score
dt_gridsearch.best_estimator_ # Tuned model

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
18,24.128553,1.514224,1.015366,0.148566,gini,30,sqrt,70,"{'criterion': 'gini', 'max_depth': 30, 'max_fe...",0.548694,0.562929,0.545834,0.566883,0.568079,0.558484,0.009362,1
21,32.685093,0.899370,1.261593,0.068304,gini,30,3,100,"{'criterion': 'gini', 'max_depth': 30, 'max_fe...",0.550359,0.563013,0.548036,0.563851,0.560727,0.557197,0.006652,2
20,24.675079,0.964299,0.909737,0.049459,gini,30,3,70,"{'criterion': 'gini', 'max_depth': 30, 'max_fe...",0.555063,0.558506,0.544469,0.566430,0.559307,0.556755,0.007169,3
19,34.305946,1.142679,1.356387,0.086663,gini,30,sqrt,100,"{'criterion': 'gini', 'max_depth': 30, 'max_fe...",0.552237,0.563367,0.543277,0.563276,0.558607,0.556153,0.007615,4
22,32.470144,0.287907,0.747786,0.015929,gini,30,6,70,"{'criterion': 'gini', 'max_depth': 30, 'max_fe...",0.550349,0.555718,0.544622,0.558209,0.558543,0.553488,0.005315,5
23,42.862380,0.356352,0.836325,0.116956,gini,30,6,100,"{'criterion': 'gini', 'max_depth': 30, 'max_fe...",0.549919,0.552283,0.539900,0.565691,0.557393,0.553037,0.008508,6
16,37.572352,1.566282,0.842004,0.051086,gini,20,6,70,"{'criterion': 'gini', 'max_depth': 20, 'max_fe...",0.450392,0.440077,0.437491,0.454290,0.450456,0.446541,0.006540,7
17,52.251724,0.913447,1.205651,0.106059,gini,20,6,100,"{'criterion': 'gini', 'max_depth': 20, 'max_fe...",0.441278,0.446985,0.430636,0.452314,0.448662,0.443975,0.007560,8
15,31.749335,1.398350,1.374073,0.260220,gini,20,3,100,"{'criterion': 'gini', 'max_depth': 20, 'max_fe...",0.435163,0.438610,0.429895,0.450164,0.448291,0.440425,0.007728,9
13,31.711588,0.600028,1.405193,0.155159,gini,20,sqrt,100,"{'criterion': 'gini', 'max_depth': 20, 'max_fe...",0.434520,0.432400,0.423770,0.453535,0.438583,0.436562,0.009770,10


{'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'sqrt',
 'n_estimators': 70}

0.5584839793281684

RandomForestClassifier(max_depth=30, n_estimators=70)

### Evaluate the performance of the tuned model.

In [39]:
scores = cross_validate(dt_gridsearch.best_estimator_, X_train, y_train,
 scoring=['f1'], cv = 5, return_train_score= False, n_jobs=-1)
scores_df = pd.DataFrame(scores)
scores_df
scores_df.mean()

,fit_time,score_time,test_f1
0,15.897679,0.787353,0.556678
1,16.086487,0.769650,0.559661
2,15.716957,0.788883,0.543210
3,15.938294,0.790605,0.569373
4,15.881673,0.806188,0.566084


fit_time      15.904218
score_time     0.788536
test_f1        0.559001
dtype: float64

In [40]:
scores = cross_validate(dt_gridsearch.best_estimator_, X_test, y_test,
 scoring=['f1'], cv = 5, return_train_score= False, n_jobs=-1)
scores_df = pd.DataFrame(scores)
scores_df
scores_df.mean()

,fit_time,score_time,test_f1
0,7.694508,0.288071,0.539359
1,7.512118,0.266504,0.528810
2,7.493896,0.301454,0.545559
3,7.523159,0.272158,0.544881
4,7.552070,0.274501,0.541245


fit_time      7.555150
score_time    0.280537
test_f1       0.539971
dtype: float64

# <font color='orange'>Gradient Boosting</font>

**To do**:
- Fit gradient boosting with default hyperparameters to the train data
- Generate the classification reports on the train and test data.
- Determine the f1-score by using K-fold CV on the train data.
- Tune the model hyperparameters.
- Determine the f1-score of the tuned model by using K-fold CV on the whole dataset. 

## Fit Gradient boosting model

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gd = GradientBoostingClassifier()
clf_gd.fit(X_train,y_train)

GradientBoostingClassifier()

## Evaluate performance

### Train and test performance

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_train, clf_gd.predict(X_train)))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93    154666
           1       0.83      0.00      0.01     21734

    accuracy                           0.88    176400
   macro avg       0.85      0.50      0.47    176400
weighted avg       0.87      0.88      0.82    176400



In [43]:
print(classification_report(y_test, clf_gd.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93     66338
           1       0.66      0.00      0.01      9262

    accuracy                           0.88     75600
   macro avg       0.77      0.50      0.47     75600
weighted avg       0.85      0.88      0.82     75600



### K-fold Cross-validation

In [44]:
scores_gd = cross_validate(clf_gd, X_train, y_train,
 scoring=['f1'], cv = 5, return_train_score= False, n_jobs=-1)
scores_df_gd = pd.DataFrame(scores)
scores_df_gd
scores_df_gd.mean()

,fit_time,score_time,test_f1
0,7.694508,0.288071,0.539359
1,7.512118,0.266504,0.528810
2,7.493896,0.301454,0.545559
3,7.523159,0.272158,0.544881
4,7.552070,0.274501,0.541245


fit_time      7.555150
score_time    0.280537
test_f1       0.539971
dtype: float64

## Model Tuning

- Perform Grid-search CV on `max_depth`, `n_estimators`, `max_features`, `learning_rate` by using F1
as the scoring metric.
- Show the tuned hyperparameter sets and the corresponding scores.
- Perform K-fold CV on the whole dataset using the tuned hyperparameters and compare the performance
results with the untuned model.

### Grid-search CV

In [45]:
# Specify the value of features to tune here.
param_grid = {
 'max_depth': [10 , 20, 30],
 'n_estimators': [70, 100, 120],
 'max_features': ["sqrt", 3, 6],
 'learning_rate': [0.1, 0.3, 0.4],
}
dt_gridsearch_gd = GridSearchCV(GradientBoostingClassifier(), param_grid, scoring='f1', verbose=True, n_jobs=-1)
dt_gridsearch_gd.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.4],
                         'max_depth': [10, 20, 30],
                         'max_features': ['sqrt', 3, 6],
                         'n_estimators': [70, 100, 120]},
             scoring='f1', verbose=True)

In [46]:
pd.DataFrame(dt_gridsearch_gd.cv_results_).sort_values('mean_test_score', ascending=False)
dt_gridsearch_gd.best_params_ # Tuned hyperparameters
dt_gridsearch_gd.best_score_ # Best score
dt_gridsearch_gd.best_estimator_ # Tuned model

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
80,305.253961,13.177381,0.767998,0.123701,0.4,30,6,120,"{'learning_rate': 0.4, 'max_depth': 30, 'max_f...",0.552479,0.558420,0.547306,0.564823,0.563237,0.557253,0.006575,1
40,425.967081,18.459478,1.806061,0.307765,0.3,20,3,100,"{'learning_rate': 0.3, 'max_depth': 20, 'max_f...",0.552700,0.557527,0.545766,0.564589,0.563990,0.556914,0.007091,2
44,494.678395,14.260111,1.881862,0.235571,0.3,20,6,120,"{'learning_rate': 0.3, 'max_depth': 20, 'max_f...",0.549117,0.557812,0.550405,0.565510,0.561503,0.556869,0.006307,3
70,401.302986,10.405293,1.547086,0.235414,0.4,20,6,100,"{'learning_rate': 0.4, 'max_depth': 20, 'max_f...",0.550879,0.556721,0.548150,0.565329,0.562157,0.556647,0.006499,4
79,369.488126,8.512664,1.009338,0.082123,0.4,30,6,100,"{'learning_rate': 0.4, 'max_depth': 30, 'max_f...",0.550778,0.556280,0.546703,0.564932,0.564495,0.556638,0.007262,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,47.926614,2.659123,0.407583,0.050263,0.1,10,sqrt,100,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.317111,0.333109,0.313476,0.334064,0.333672,0.326287,0.009054,77
7,64.871366,3.418722,0.389209,0.065555,0.1,10,6,100,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.310447,0.327366,0.320382,0.322092,0.339203,0.323898,0.009411,78
6,46.241005,2.823842,0.213556,0.013803,0.1,10,6,70,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.241893,0.254009,0.222349,0.261014,0.255549,0.246963,0.013801,79
0,33.884847,3.237097,0.286690,0.037238,0.1,10,sqrt,70,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.224762,0.241803,0.228582,0.258112,0.240332,0.238718,0.011711,80


{'learning_rate': 0.4, 'max_depth': 30, 'max_features': 6, 'n_estimators': 120}

0.5572529342064892

GradientBoostingClassifier(learning_rate=0.4, max_depth=30, max_features=6,
                           n_estimators=120)

### Evaluate the performance of the tuned model.

In [47]:
scores = cross_validate(dt_gridsearch_gd.best_estimator_, X_train, y_train,
 scoring=['f1'], cv = 5, return_train_score= False, n_jobs=-1)
scores_df_gd = pd.DataFrame(scores)
scores_df_gd
scores_df_gd.mean()

,fit_time,score_time,test_f1
0,173.553862,0.883151,0.551427
1,171.694958,0.925808,0.555299
2,175.025432,0.685989,0.546500
3,177.821348,0.795521,0.568121
4,174.176378,0.849319,0.559873


fit_time      174.454396
score_time      0.827957
test_f1         0.556244
dtype: float64

In [48]:
scores = cross_validate(dt_gridsearch_gd.best_estimator_, X_test, y_test,
 scoring=['f1'], cv = 5, return_train_score= False, n_jobs=-1)
scores_df_gd = pd.DataFrame(scores)
scores_df_gd
scores_df_gd.mean()

,fit_time,score_time,test_f1
0,65.320732,0.341397,0.523723
1,65.313655,0.337971,0.514388
2,65.737094,0.263178,0.530672
3,65.162094,0.359375,0.524852
4,66.217520,0.232095,0.523398


fit_time      65.550219
score_time     0.306803
test_f1        0.523407
dtype: float64